In [2]:


!pip install pandas


ModuleNotFoundError: No module named 'pandas'

In [ ]:
import pandas as pd
import numpy as mp
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")